In [15]:
from neo4j import GraphDatabase
import os

URI = "bolt://localhost:7687"
USERNAME = "neo4j"
# 如果环境变量没设置，就默认用 "12345678"
PASSWORD = os.getenv("NEO4J_PASSWORD", "12345678")  

class Neo4jDemo:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        print("✅ Neo4j 驱动已初始化。")

    def close(self):
        self._driver.close()
        print(" Neo4j 连接已关闭。")

    # --- 3. 写入操作：创建节点和关系 ---
    def create_and_connect_data(self):
        with self._driver.session() as session:
            # 1. 清理数据
            session.run("MATCH (n) DETACH DELETE n")
            
            # 2. 创建节点和关系
            cypher_query = """
            CREATE (p1:Person {name: 'Alice', born: 1980})
            CREATE (p2:Person {name: 'Bob', born: 1990})
            CREATE (p3:Person {name: 'Charlie', born: 1985})
            CREATE (m1:Movie {title: 'The Matrix', released: 1999})
            CREATE (m2:Movie {title: 'Inception', released: 2010})
            CREATE (p1)-[:ACTED_IN {roles: ['Trinity']}]->(m1)
            CREATE (p2)-[:ACTED_IN {roles: ['Arthur']}]->(m2)
            CREATE (p1)-[:KNOWS]->(p2)
            CREATE (p3)-[:KNOWS]->(p1)
            RETURN p1, p2, m1, m2
            """
            result = session.run(cypher_query)
            summary = result.consume()
            print(f"✨ 写入操作完成。创建了 {summary.counters.nodes_created} 个节点, "
                  f"{summary.counters.relationships_created} 个关系。")
            return summary

    # --- 4. 读取操作：查询数据 ---
    def get_people_who_know_alice(self):
        cypher_query = """
        MATCH (alice:Person {name: 'Alice'})<-[:KNOWS]-(p:Person)
        RETURN p.name AS friendName, p.born AS friendBorn
        """
        with self._driver.session() as session:
            result = session.run(cypher_query)
            
            people_list = []
            for record in result:
                people_list.append({
                    "Name": record["friendName"], 
                    "Born": record["friendBorn"]
                })
            
            print("\n🔍 查询结果：认识 Alice 的人：")
            for person in people_list:
                print(f"  - 姓名: {person['Name']}, 出生年份: {person['Born']}")
            
            return people_list


if __name__ == "__main__":
    db = Neo4jDemo(URI, USERNAME, PASSWORD)
    try:
        db.create_and_connect_data()
        db.get_people_who_know_alice()
    except Exception as e:
        print(f"\n❌ 发生错误，请检查 Neo4j 服务器是否运行或连接信息是否正确：{e}")
    finally:
        db.close()

✅ Neo4j 驱动已初始化。
✨ 写入操作完成。创建了 5 个节点, 4 个关系。

🔍 查询结果：认识 Alice 的人：
  - 姓名: Charlie, 出生年份: 1985
 Neo4j 连接已关闭。


In [ ]:
import json
from neo4j import GraphDatabase
import os

# Neo4j 连接信息
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
# 如果环境变量没设置，就默认用 "12345678"
PASSWORD = os.getenv("NEO4J_PASSWORD", "12345678")

# JSON 文件路径
COURSE_JSON_PATH = "data/course/big_data.json"

class Neo4jDemo:
    def __init__(self, uri, user, password):
        # 尝试创建驱动
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        # 检查连接（可选，但推荐）
        self._driver.verify_connectivity()
        print("✅ Neo4j 驱动已初始化并连接成功。")

    def close(self):
        self._driver.close()
        print(" Neo4j 连接已关闭。")

    # 递归处理课程 JSON 的辅助函数
    # session: Neo4j 事务会话
    # node_data: 当前层级节点的数据字典 (e.g., {"name": "...", "flag": "...", "children": [...]})
    # parent_name: 父节点的名称 (用于建立 HAS_CHILD 关系)
    def _process_node_recursive(self, session, node_data, parent_name=None):
        node_name = node_data.get("name")
        flag = node_data.get("flag", "0")
        resource_paths = node_data.get("resource_path")
        
        if not node_name:
            # 根节点可能只有 root_name
            return

        # 1. 创建当前 CourseNode 节点 (并处理 flag 属性)
        # MERGE 确保幂等性：如果节点不存在则创建，如果存在则什么也不做 (但这里我们是清空后重新导入，用 CREATE 也可以，MERGE 更稳健)
        session.run("""
            MERGE (c:CourseNode {name: $name})
            SET c.flag = $flag
            RETURN c
        """, name=node_name, flag=flag)
        
        # 2. 如果有父节点，创建 HAS_CHILD 关系
        if parent_name:
            session.run("""
                MATCH (p:CourseNode {name: $parent_name})
                MATCH (c:CourseNode {name: $child_name})
                MERGE (p)-[:HAS_CHILD]->(c)
            """, parent_name=parent_name, child_name=node_name)

        # 3. 如果有 resource_path，创建 Resource 节点和 USES_RESOURCE 关系
        if isinstance(resource_paths, list) and resource_paths:
            for path in resource_paths:
                # 创建 Resource 节点
                session.run("""
                    MERGE (r:Resource {path: $path})
                    RETURN r
                """, path=path)
                
                # 创建 USES_RESOURCE 关系
                session.run("""
                    MATCH (c:CourseNode {name: $course_name})
                    MATCH (r:Resource {path: $path})
                    MERGE (c)-[:USES_RESOURCE]->(r)
                """, course_name=node_name, path=path)

        # 4. 递归处理下一层级
        # 统一处理可能的 children, grandchildren, great-grandchildren 等键
        child_keys = ["children", "grandchildren", "great-grandchildren"]
        for key in child_keys:
            if key in node_data and isinstance(node_data[key], list):
                for child_node_data in node_data[key]:
                    self._process_node_recursive(session, child_node_data, parent_name=node_name)


    def import_course_data(self, json_file_path):
        # 读取 JSON 文件
        try:
            with open(json_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except FileNotFoundError:
            print(f"❌ 错误：文件未找到 at {json_file_path}")
            return
        except json.JSONDecodeError:
            print("❌ 错误：JSON 文件格式不正确。")
            return

        with self._driver.session() as session:
            # --- 1. 清理数据 ---
            print("⏳ 正在清空数据库...")
            session.run("MATCH (n) DETACH DELETE n")
            print("🧹 数据库已清空。")
            
            # --- 2. 处理根节点 ---
            root_name = data.get("root_name")
            if not root_name:
                print("❌ 错误：JSON 中缺少 'root_name' 字段。")
                return

            # 创建根节点
            session.run("MERGE (c:CourseNode {name: $name}) SET c.flag = '1' RETURN c", name=root_name)

            # --- 3. 递归处理子节点 ---
            print("⏳ 正在导入课程数据...")
            children_data = data.get("children", [])
            for child_data in children_data:
                self._process_node_recursive(session, child_data, parent_name=root_name)
            
            # 统计导入结果
            stats_query = """
                MATCH (c:CourseNode) WITH count(c) AS courseNodes
                MATCH (r:Resource) WITH courseNodes, count(r) AS resources
                MATCH ()-[rel]->() RETURN courseNodes, resources, count(rel) AS relations
            """
            result = session.run(stats_query).single()
            
            print(f"✨ 导入完成。")
            if result:
                print(f"   - 创建了 {result['courseNodes']} 个 CourseNode 节点")
                print(f"   - 创建了 {result['resources']} 个 Resource 节点")
                print(f"   - 创建了 {result['relations']} 个关系 (HAS_CHILD, USES_RESOURCE)")
            else:
                 print("   - 未能获取统计信息。")
            
            return True


if __name__ == "__main__":
    db = None
    try:
        db = Neo4jDemo(URI, USERNAME, PASSWORD)
        # 运行导入逻辑
        db.import_course_data(COURSE_JSON_PATH)

    except Exception as e:
        print(f"\n❌ 发生错误，请检查 Neo4j 服务器是否运行或连接信息是否正确：{e}")
    finally:
        if db:
            db.close()

✅ Neo4j 驱动已初始化并连接成功。
⏳ 正在清空数据库...
🧹 数据库已清空。
⏳ 正在导入课程数据...
✨ 导入完成。
   - 创建了 232 个 CourseNode 节点
   - 创建了 2 个 Resource 节点
   - 创建了 242 个关系 (HAS_CHILD, USES_RESOURCE)
 Neo4j 连接已关闭。
